---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [2]:
def get_list_of_university_towns():
    data=open('university_towns.txt')
    Collegetowns=data.readlines()
    university_towns=list()
    for line in Collegetowns:
        if 'edit' in line:
            state=line.split('[')[0]
            continue
        RegionName=line.split('(')[0].rstrip()
        university_towns.append([state, RegionName])
    university_towns=pd.DataFrame(university_towns)
    university_towns.columns=['State','RegionName']
    return university_towns

In [1]:
def get_recession_start():
    GDP=pd.read_excel('gdplev.xls',skiprows=5)
    GDP=GDP[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']]
    GDP=GDP.dropna()
    GDP=GDP[GDP['Unnamed: 4']>='2000q1']
    GDP.columns=['period','GDP']
    lis=list(GDP['GDP'])
    del lis[0]
    lis.append(None)
    GDP['diff']=lis
    GDP['diff']=GDP['GDP']-GDP['diff']
    lis = list()
    n=len(GDP)
    st=0
    for i in range(n-1):
        a = GDP.iloc[i]
        b = GDP.iloc[i+1]
        if a['diff'] > 0 and b['diff']> 0:
            if st == 0 or i > (st+3):
                st = i
                lis.append(b['period'])
    return lis[0]

In [ ]:
def get_recession_end():
    GDP=pd.read_excel('gdplev.xls',skiprows=5)
    GDP=GDP[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']]
    GDP=GDP.dropna()
    GDP=GDP[GDP['Unnamed: 4']>='2000q1']
    GDP.columns=['period','GDP']
    lis=list(GDP['GDP'])
    del lis[0]
    lis.append(None)
    GDP['diff']=lis
    GDP['diff']=GDP['GDP']-GDP['diff']
    a=get_recession_start()
    GDP=GDP[GDP['period']>=a]
    n=len(GDP)
    st=0
    end = list()
    for i in range(n-1):
        a = GDP.iloc[i]
        b = GDP.iloc[i+1]
        if a['diff'] < 0 and b['diff']< 0:
            if st ==  0:
                st = i
                end.append(GDP.iloc[i+2]['period'])
    return end[0]

In [ ]:
def get_recession_bottom():
    GDP=pd.read_excel('gdplev.xls',skiprows=5)
    GDP=GDP[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']]
    GDP=GDP.dropna()
    GDP=GDP[GDP['Unnamed: 4']>='2000q1']
    GDP.columns=['period','GDP']
    lis=list(GDP['GDP'])
    del lis[0]
    lis.append(None)
    GDP['diff']=lis
    GDP['diff']=GDP['GDP']-GDP['diff']
    a=get_recession_start()
    b=get_recession_end()
    GDP=GDP[(GDP['period']>=a)]
    GDP=GDP[GDP['period']<=b]
    return GDP.set_index('period')['GDP'].idxmin()

In [ ]:
def convert_housing_data_to_quarters():
    import pandas as pd
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 
              'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National',
              'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 
              'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana',
              'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 
              'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine',
              'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan',
              'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
              'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 
              'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands',
              'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut',
              'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana',
              'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 
              'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 
              'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
              'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
              'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
              'ND': 'North Dakota', 'VA': 'Virginia'}
    Home=pd.read_csv('City_Zhvi_AllHomes.csv')
    Homeprice=Home[Home.columns[Home.columns >= '2000-01']]
    Homeprice=Homeprice[Homeprice.columns[Homeprice.columns <= '2016-10']]
    lis = list()
    li = list()
    for i in range(len(Homeprice.columns)//3):
        v1 = 3*i
        v2 = v1+3
        y=i//4
        year=y+2000
        if i%4==0:
            q = 'q1'
        elif i%4==1:
            q = 'q2'
        elif i%4==2:
            q = 'q3'
        else : q= 'q4'
        a='{}{}'.format(year,q)
        li.append(a)
        lis.append(Homeprice.iloc[:,v1:v2].mean(axis=1))
    kk=pd.DataFrame(lis)
    kk=kk.T
    kk.columns=li
    kk['2016q3']=(Home['2016-07']+Home['2016-08'])/2
    Home=Home[['State','RegionName']]
    Home=Home.replace({'State':states})
    House=pd.merge(Home,kk,left_index=True,right_index=True)
    return House.set_index(['State','RegionName'])

In [1]:
def run_ttest():
    rb=get_recession_bottom()
    df=convert_housing_data_to_quarters()
    recession_start=get_recession_start()
    GDP=pd.read_excel('gdplev.xls',skiprows=5)
    GDP=GDP[['Unnamed: 4','GDP in billions of chained 2009 dollars.1']]
    GDP=GDP.dropna()
    GDP.columns=['period','GDP']
    qrb=GDP[GDP['period']<=recession_start].iloc[-1]['period']
    df=df[[qrb,rb]]
    df['price_ratio']=df['2008q3']/df['2009q2']
    df2 = get_list_of_university_towns()
    lis =list()
    for i in range(len(df2)):
        st = df2['State'].iloc[i]
        rn = df2['RegionName'].iloc[i]
        lis.append((rn, st))
    df=df.reset_index()
    lis2 = list()
    for i in range(len(df)):
        ab=df.iloc[i]['RegionName']
        cd=df.iloc[i]['State']
        lis2.append((ab, cd))
    df['New']=lis2
    lis3 = list()
    for i in range(len(df)):
        if df['New'].iloc[i] in lis:
            lis3.append(1)
        else : 
            lis3.append(0)
    df['university']=lis3
    non=df[df['university']==0]
    univ=df[df['university']==1]
    from scipy import stats
    import numpy as np
    ttes=stats.ttest_ind(non['price_ratio'], univ['price_ratio'], nan_policy='omit')
    p=ttes[1]
    if p <0.01:
        different = True
    else :
        different = False
    if non['price_ratio'].mean() <= univ['price_ratio'].mean():
        better = 'non-university town'
    else : better = 'university town'
    return (different, p, better)